Visualization of the results of the model
=================================


A GAMCR model has two main methods to produce results:

* `.predict_streamflow(matJ)`: to get the hydrograph

* `.predict_transfer_function(X)`: to get the transfer functions (RRDs)

# 1. Simple visualization

Below we provide is simple code to show the observed and predicted hydrographs, and the predicted average NRF transfer function.

In [ ]:
import GAMCR

site= 'flashy'
# You should change this line specifying the path of your model
model.load_model(os.path.join(root,'{0}/{0}_best_model.pkl'.format(site)))
X, matJ, y, timeyear, dates = model.load_data(os.path.join(root, '{0}/data/'.format(site)), max_files=20)

yhat = model.predict_streamflow(matJ)
plt.figure()
plt.plot(yhat, label='estimation')
plt.plot(y, label='ground truth', linestyle='--')
plt.plot([],[], color='white', label='NSE: {0}'.format(GAMCR.nse(y, yhat)))
plt.title(site)
plt.legend()
plt.show()

a = np.mean(model.gam._modelmat(X), axis=0)
smooth_P = np.kron( GAMCR.build_custom_matrix(model.L), np.dot(a.reshape(-1,1), a.reshape(1,-1)) )

coeffs = model.gam.get_coeffs().reshape(-1)
print( coeffs.T @ smooth_P @ coeffs)

plt.figure()
H = model.predict_transfer_function(X)
plt.figure()
plt.plot(np.array([i/24 for i in range(24*3)]), H.mean(axis=0)[:24*3], label='estimation')
plt.title(site)
plt.legend()
plt.show()

# 2. Advanced analysis of the results

By launching the script `compute_statistics.py`, you can compute different information on the learned transfer functions, such as:
- the global average NRF/RDD/weighted RRD
- the average NRF/RRD/weighted RRD over some ensembles (you can stratify either by precipitation intensity, antecendent wetness or by both)
- the area, mean, peak and peak lag of the transfer function over different ensembles for the precipitation intensity

## Note that if you want to compare GAMCR with ERRA, you should first launch the script ERRA_experiments.R located in the folder ./experiments/ERRA/

If ERRA data is available, the ensembles will be automatically set to the ones used by ERRA to allow a fair comparison. Otherwise, we can define yourself the ensembles.

For ease of use, we save the global (and stratified) NRF/RDD/weighted RRD in a CSV file `NRF_RRD.csv`, located in the `results` folder of the site under study. In this file, the columns named `NRF`, `RRD`, and `weighted_avg_RRD` correspond to the average over all time points. The columns named `NRF_group_{k}`, `RRD_group_{k}`, and `weighted_avg_RRD_group_{k}` correspond to the average over the time points for the group with index k. To determine the precipitation and/or antecedent wetness ranges corresponding to a group, you should open the file `groups.csv`.

### ➡️ Run the Notebook: `figure_real_data_paper.ipynb`

A set of tools has already been implemented in GAMCR. By opening the notebook `figures_paper.ipynb` in the `data_and_visualization` folder, you can view the visualizations available using GAMCR.

![Screen shot of NRF averaged over different ranges of precipitation intensity.](../../_static/nb_visualization.png)